In [1]:
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("merge_entities")
nlp.add_pipe("merge_noun_chunks")
import json
import re

In [41]:
class question_matcher:
    def __init__(self):
        self.patterns= []
        self.q_json = self.get_questions_json()
        self.matcher = Matcher(nlp.vocab)

    def get_questions_json(self):
        with open('questions.json', 'r') as f:
            pat = json.load(f)
        return pat
    
    def define_keyword(self,question:str):
        for q in self.q_json:
            if q['keyword'] in question.lower():
                keyword = q['keyword']
                index = q['qid']
        return keyword, index

    def get_focus(self,question:str,ind,pattern):
        focus =''
        focus_pos = pattern['focus']
        token_q = [token.text for token in nlp(question)]
        i = 0
        start = 0
        for token in focus_pos[ind].split():
            if(token == 'sf'):
                start = 1
                continue
                
            if(token == 'ef'):
                start = 0
                break;
            if(start == 1):
                focus = focus + ' ' + token_q[i]
            i = i + 1
        return focus
    
    def re_get_focus(self,question:str,ind,pattern):
        question_token = [token.text for token in nlp(question)]
        focus = ''
        focus_pos = pattern['focus']
        start = 0
        i = 0
        for token in focus_pos[ind].split('\s*'):
                if(token == 'sf'):
                    start = 1
                    i = i - 1
                    continue
                    
                if(token == 'ef'):
                    start = 0
                    break;
                if(start == 1):
                    focus = focus + ' ' + question_token[i]
                i = i + 1
        print(focus)

    def match_pattern(self,question):
        q_pos = [token.pos_ for token in nlp(question)]
        keyword, index = self.define_keyword(question)
        for pattern in self.q_json[index]['patterns']:
            pos = self.pattern_to_pos(pattern['pos'])
            self.matcher.add(p, pos)
            doc = nlp(question)
            matches = self.matcher(doc)
            if matches != []:
                for match_id, start, end in matches:
                    string_id = nlp.vocab.strings[match_id]
                    span = doc[start:end]
                    print(string_id)
                    print(q_pos)
                    print(pattern['pos'])
                    print(match_id)
                    # if((span.text == question) and (q_pos == pattern['pos'][match_id].split())):
                    #     print(match_id, string_id, start, end, span.text)
                    #     print(self.get_focus(question, match_id,pattern))
                    #     return pattern['pattern']
            else:
                print('fail')

    def re_match_pattern(self,question):
        doc = nlp(question)
        ltoken = [token.text for token in nlp(question)]
        q_pattern = ''
        for token in doc:
            q_pattern = q_pattern + token.pos_ + ' ' 
        keyword, index = self.define_keyword(question)
        for pattern in self.q_json[index]['patterns']:
            for pos in pattern['pos']:
                regexpos  = f'({pos})'
                matches = re.fullmatch(regexpos, q_pattern)
                if matches != None:
                    print('Matched!')
                    print(matches.string)
                    print(pattern['pattern'])
                    self.re_get_focus(question,index,pattern)


    def on_match(self,matcher, doc, id, matches):
        print('Matched!', matches)


    def pattern_to_pos(self, patterns:list):
        pos = []
        pattern = []
        for p in patterns:
            
            itr = []
            pattern.append(p)
            for token in p.split():
                if token != '*':
                    itr.append({'POS':token})
                else:
                    itr.append({})
            pos.append(itr)
        return pos
qm = question_matcher()


In [43]:
qm.re_match_pattern('who is mohammad')

Matched!
PRON AUX PROPN 
who [is,was] [NAME]
 mohammad


In [12]:
with open('questions.json', 'r') as f:
    patt = json.load(f)



In [30]:
question = """Who is the president of united states"""
doc = nlp(question)
ltoken = [token.text for token in nlp(question)]
q_pattern = ''
for token in doc:
    q_pattern = q_pattern + ' ' + token.pos_ 

pat = patt[0]['patterns'][0]['pos'][1]
focus =patt[0]['patterns'][0]['focus'][1]
f =''
regpat = f'({pat})'
print(regpat)
matches = re.fullmatch(regpat, q_pattern)

if matches != None:
    start = 0
    i = 0
    for token in focus.split('\s*'):
            if(token == 'sf'):
                start = 1
                i = i - 1
                continue
                
            if(token == 'ef'):
                start = 0
                break;
            if(start == 1):
                f = f + ' ' + ltoken[i]
            i = i + 1
    print(f)
        

(\s*PRON\s*AUX\s*NOUN\s*ADP\s*PROPN)
 the president of united states


In [3]:
doc = nlp("Who is mohammad dareer")
print( [token.pos_ for token in doc])
print( [token.text for token in doc])

['PRON', 'AUX', 'NOUN']
['Who', 'is', 'mohammad dareer']


In [6]:
qm.re_match_pattern('who is mohammad dareer')

In [ ]:
doc = nlp("who is the president of united states")
print( [token.pos_ for token in doc])
print( [token.text for token in doc])

['PRON', 'AUX', 'NOUN', 'ADP', 'PROPN']
['who', 'is', 'the president', 'of', 'united states']


In [12]:
qm.match_pattern('who is the president of united states')

'who [is,was] [NAME]'

In [13]:
qm.match_pattern('give me images that contains red color')

IndexError: list index out of range